In [6]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_finance
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.endpoints.pricing import PricingInfo
import datetime
from statistics import mean
from statistics import median
import calendar
import datetime
from math import floor
import schedule
import time
import threading
import json
import statistics

In [2]:
# OANDA API v20の口座IDとAPIトークン
accountID = "101-009-12609641-001"
access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")

In [3]:
# APIから取得したレートをPandasのDataFrameへ
def to_dataframe(r):
    data = []
    for raw in r.response['candles']:
        data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    # リストからPandas DataFrameへ変換
    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    return df

In [4]:
def get_data(count, gran, year, month, day, hour, minute, second):
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    _from = datetime.datetime(year=year, month=month, day=day,
                              hour=hour, minute=minute, second=second).strftime(fmt)
    params = {
        "count": count,
        "granularity": gran,
        'from': _from,
#         'dailyAlignment': 0
    }
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    return df

In [5]:
# 第何何曜日（第nX曜日）の日付を返す関数
def get_day_of_nth_dow(year, month, nth, dow):
    '''dow: Monday(0) - Sunday(6)'''
    if nth < 1 or dow < 0 or dow > 6:
        return None

    first_dow, n = calendar.monthrange(year, month)
    day = 7 * (nth - 1) + (dow - first_dow) % 7 + 1

    return day if day <= n else None

In [6]:
def get_data_super(start, end, gran):
    # xxxx-xx-xx～xxxx-xx-xx の期間のデータを取得してデータフレームを返す
    s_year, s_month, s_day, s_hour, s_minute, s_second = map(int, start.split('-'))
    e_year, e_month, e_day, e_hour, e_minute, e_second = map(int, end.split('-'))
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    from_ = datetime.datetime(year=s_year, month=s_month, day=s_day,
                              hour=s_hour, minute=s_minute, second=s_second).strftime(fmt)
    to_ = datetime.datetime(year=e_year, month=e_month, day=e_day,
                            hour=e_hour, minute=e_minute, second=e_second).strftime(fmt)
    df = pd.DataFrame()
    # 5000個制限に引っかからなければこっちの処理
    try:
        params = {
            "granularity": gran,
            'from': from_,
            'to': to_
        }
        r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
        api.request(r)
        df = to_dataframe(r)
    # 引っかかればこっちの処理
    except:
        i = True
        while i:
            df_tmp = get_data(count=5000, gran=gran, year=s_year, month=s_month, day=s_day,
                             hour=s_hour, minute=s_minute, second=s_second)
            df = pd.concat([df, df_tmp], ignore_index=True)
            s_year = df_tmp.iat[df_tmp.tail(1).index[0], 0].year
            s_month = df_tmp.iat[df_tmp.tail(1).index[0], 0].month
            s_day = df_tmp.iat[df_tmp.tail(1).index[0], 0].day
            s_hour = df_tmp.iat[df_tmp.tail(1).index[0], 0].hour
            s_minute = df_tmp.iat[df_tmp.tail(1).index[0], 0].minute
            s_second = df_tmp.iat[df_tmp.tail(1).index[0], 0].second
#             print('{}-{}-{}-{}-{}-{}'.format(s_year, s_month, s_day, s_hour, s_minute, s_second))
            # 5000個ずつ取得してエンド超えたら止める
            if s_year > e_year:
                i = False
        # 止めたらはみ出した分をここで切り落とす
        for i in range(len(df.index) - 1000, len(df.index)):
            if df.iat[i, 0].year == e_year and df.iat[i, 0].month == e_month and df.iat[i, 0].day == e_day:
                df = df[0:i+1]
                break
    return df

In [7]:
def set_profit(tmp_i, i, pos, entry_rate, exit_rate, assets, kly_rate, df, pos_df, pos_count):
    fee = 0.008
    pos_count+=1
    if pos == 'entry_long':
        pips = (exit_rate - entry_rate - fee) * 100
#       print('L利確 entry_rate: {}, exit_rate: {}, pips: {}'.format(entry_rate, exit_rate, pips))

        # ケリー基準で Lot を決める
        kly = assets * kly_rate
        lot = floor(kly / exit_rate)
        if lot < 1000:
            lot = 1000
        elif lot > 3000000:
            lot = 3000000
        profit = lot * 25 * pips
        assets = round(assets + profit, 4)
        tmp_se = pd.Series( [lot, df.iat[tmp_i,0], entry_rate, df.iat[i+1,0], exit_rate, pips, profit, assets ], index=pos_df.columns )
        pos_df = pos_df.append( tmp_se, ignore_index=True )
        return assets, pos_df, pos_count
    else:
        pips = (entry_rate - exit_rate - fee) * 1000
#       print('S利確 entry_rate: {}, exit_rate: {}, pips: {}'.format(entry_rate, exit_rate, pips))

        # ケリー基準で Lot を決める
        kly = assets * kly_rate
        lot = floor(kly / exit_rate)
        if lot < 1000:
            lot = 1000
        elif lot > 3000000:
            lot = 3000000
        profit = lot * 25 * pips
        assets = round(assets + profit, 4)
        tmp_se = pd.Series( [lot, df.iat[tmp_i,0], entry_rate, df.iat[i+1,0], exit_rate, pips, profit, assets ], index=pos_df.columns )
        pos_df = pos_df.append( tmp_se, ignore_index=True )
        return assets, pos_df, pos_count

In [8]:
# バックテスト
def back_test(K, start, end, gran, ref, seed_money, kly_rate):
    K = K # レンジ幅係数
    range_ = 0
    range_list = []
    ave_range = 0
    least_5 = 0
    pos = 'None'
    entry_rate = 0
    pos_count = 0
    ref = ref
    assets = seed_money
    lot = 0
    kly_rate = kly_rate
    pips_tmp = 0
    los_count = 0
    los_set = -1.5
    tmp_i = 0
    
    df = get_data_super(start, end, gran)
    pos_df = pd.DataFrame(columns=['lot', 'entry_date', 'entry_rate', 'exit_date', 'exit_rate', 'pips', 'profit', 'assets'])
    
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12609641-001"
    access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")

    # 実行
    for i in range(len(df.index) - 1):
        rate = df.iat[i, 5] # 当日の終値
        next_rate = df.iat[i+1, 2] # 次の足の始値（エントリー価格）
        range_ = abs(round(df.iat[i, 3] - df.iat[i, 4], 3)) # |High - Low| レンジ幅
        range_list.append(range_)
    
        if i > ref:
            least_m = round(mean(range_list[i-ref-1:i-1]) * K, 3) # 直近5個のレンジ幅の平均
            red_range = round(df.iat[i, 3] - df.iat[i, 2], 3) # High - Open 赤幅(上げ幅)
            blue_range = round(df.iat[i, 2] - df.iat[i, 4], 3) # Open - Low 青幅(下げ幅)
            # print('least_m: {} open_high: {} open_low: {}'.format(least_m, open_high, open_low))

            # 上昇トレンドと判断しロング
            if df.iat[i, 2] < df.iat[i, 5] and least_m < red_range: # 始値より終値の方が高く、5個平均よりもレンジが大きいとき
                if pos == 'entry_long':
                    continue
                else:
                    # もし逆のポジションを持ってたら利確
                    if pos == 'entry_short':
                        exit_rate = next_rate
                        assets, pos_df, pos_count = set_profit(tmp_i, i, pos, entry_rate, exit_rate, assets, kly_rate, df, pos_df, pos_count)
                        if assets < 0:
                            print('game over')
                            break
                    
                    entry_rate = next_rate
                    pos = 'entry_long'
                    tmp_i = i+1
                    # print(pos)
        
            # 下降トレンドと判断しショート
            if df.iat[i, 5] < df.iat[i, 2] and least_m < blue_range: # 終値より始値の方が高く、5個平均よりもレンジが大きいとき
                if pos == 'entry_short':
                    continue
                else:
                    # もし逆のポジションを持ってたら利確
                    if pos == 'entry_long':
                        exit_rate = next_rate
                        assets, pos_df, pos_count = set_profit(tmp_i, i, pos, entry_rate, exit_rate, assets, kly_rate, df, pos_df, pos_count)
                        if assets < 0:
                            print('game over')
                            break
        
                    entry_rate = next_rate
                    pos = 'entry_short'
                    tmp_i = i+1
                    # print(pos)
                    
    print('posision count: {}, seed_money: {}万円, assets: {}万円'.format(pos_count, seed_money/10000, round(assets/10000, 4)))
    return pos_df, df, round(assets/10000, 4)
#     return pos_df, df

### 口座情報の取得

In [ ]:
# OANDA API v20の口座IDとAPIトークン
accountID = "101-009-12609641-001"
access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")
s = accounts.AccountSummary(accountID)
api.request(s)

### 現在のポジションを取得

In [11]:
# OANDA API v20の口座IDとAPIトークン
accountID = "101-009-12609641-001"
access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")
p = positions.OpenPositions(accountID=accountID)
api.request(p)

{'positions': [], 'lastTransactionID': '59'}

In [12]:
try:
    l_pos = int(p.response['positions'][0]['long']['units'])
    s_pos = int(p.response['positions'][0]['short']['units'])
    pos = l_pos + s_pos
except:
    pos = 0
print(pos)

0


In [13]:
        if pos == 0:
            pass
        elif pos > 0:
            position_data = { "longUnits": "ALL" }
            print('ロングを決済')
            p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
            api.request(p)
        else:
            position_data = { "shortUnits": "ALL" }
            print('ショートを決済')
            p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
            api.request(p)

### 注文

In [98]:
data = {
  "order": {
    "instrument": "USD_JPY",
    "units": "+1",
    "type": "MARKET",
    "positionFill": "DEFAULT"
  }
}
o = orders.OrderCreate(accountID, data=data)
api.request(o)
o.response

{'orderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '1',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'reason': 'CLIENT_ORDER',
  'id': '55',
  'accountID': '101-009-12609641-001',
  'userID': 12609641,
  'batchID': '55',
  'requestID': '24617943972983195',
  'time': '2019-11-22T07:32:06.276523698Z'},
 'orderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '55',
  'instrument': 'USD_JPY',
  'units': '1',
  'requestedUnits': '1',
  'price': '108.578',
  'pl': '0.0000',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '2998261.1808',
  'gainQuoteHomeConversionFactor': '1',
  'lossQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '0.0020',
  'fullVWAP': '108.578',
  'reason': 'MARKET_ORDER',
  'tradeOpened': {'price': '108.578',
   'tradeID': '56',
   'units': '1',
   'guaranteedExecutionFee': '0.0000',
   'halfSpreadCost': '0.0020',
   'initialMarginRequired': '4.3430'},
  

### 決済

In [36]:
position_data = { "longUnits": "ALL" }
p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
api.request(p)
p.response

{'longOrderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '-3',
  'timeInForce': 'FOK',
  'positionFill': 'REDUCE_ONLY',
  'reason': 'POSITION_CLOSEOUT',
  'longPositionCloseout': {'instrument': 'USD_JPY', 'units': 'ALL'},
  'id': '49',
  'accountID': '101-009-12609641-001',
  'userID': 12609641,
  'batchID': '49',
  'requestID': '24617877358095536',
  'time': '2019-11-22T03:07:24.252384609Z'},
 'longOrderFillTransaction': {'type': 'ORDER_FILL',
  'orderID': '49',
  'instrument': 'USD_JPY',
  'units': '-3',
  'requestedUnits': '-3',
  'price': '108.619',
  'pl': '0.0020',
  'financing': '0.0000',
  'commission': '0.0000',
  'accountBalance': '2998261.1548',
  'gainQuoteHomeConversionFactor': '1',
  'lossQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '0.0060',
  'fullVWAP': '108.619',
  'reason': 'MARKET_ORDER_POSITION_CLOSEOUT',
  'tradesClosed': [{'tradeID': '44',
    'units': '-1',
    'realizedPL': '-0.0

### 現在価格の取得

In [131]:
params = {"instruments": "USD_JPY"}
pi = PricingInfo(accountID=accountID, params=params)
api.request(pi)

{'time': '2019-11-22T08:00:30.496446547Z',
 'prices': [{'type': 'PRICE',
   'time': '2019-11-22T08:00:28.841642602Z',
   'bids': [{'price': '108.593', 'liquidity': 250000}],
   'asks': [{'price': '108.597', 'liquidity': 250000}],
   'closeoutBid': '108.574',
   'closeoutAsk': '108.616',
   'status': 'tradeable',
   'tradeable': True,
   'unitsAvailable': {'default': {'long': '250000', 'short': '250000'},
    'openOnly': {'long': '250000', 'short': '250000'},
    'reduceFirst': {'long': '250000', 'short': '250000'},
    'reduceOnly': {'long': '0', 'short': '0'}},
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'USD_JPY'}]}

In [132]:
pi.response['prices'][0]['asks'][0]['price']

'108.597'

### チェック

In [115]:
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12609641-001"
    access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    
    params = {'count': 7, 'granularity': 'H12'}
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    df.drop(index=6, inplace=True)

In [124]:
df

,Time,Volume,Open,High,Low,Close
0,2019-11-18 22:00:00+00:00,5501,108.682,108.802,108.462,108.774
1,2019-11-19 10:00:00+00:00,5901,108.776,108.836,108.454,108.540
2,2019-11-19 22:00:00+00:00,7365,108.546,108.584,108.349,108.426
3,2019-11-20 10:00:00+00:00,10659,108.428,108.740,108.378,108.608
4,2019-11-20 22:00:00+00:00,9979,108.594,108.674,108.280,108.529
5,2019-11-21 10:00:00+00:00,8431,108.526,108.700,108.504,108.612


In [121]:
        # 直近5個のレンジ幅の平均取得
        for i in range(5):
            range_ = abs(round(df.iat[i, 3] - df.iat[i, 4], 3)) # |High - Low|
            range_list.append(range_)
        least_m = round(mean(range_list) * 0.5, 3)
        
        red_range = round(df.iat[5, 3] - df.iat[5, 2], 3) # High - Open 赤幅(上げ幅)
        blue_range = round(df.iat[5, 2] - df.iat[5, 4], 3) # Open - Low 青幅(下げ幅)
        print('least_m: {} red_range: {} blue_range: {}'.format(least_m, red_range, blue_range))
        
        # 上昇トレンドと判断しロング
        if df.iat[5, 2] < df.iat[5, 5] and least_m < red_range: # 始値より終値の方が高く、5個平均よりもレンジが大きいとき
            print('long')
            
        # 下降トレンドと判断しショート
        if df.iat[5, 5] < df.iat[5, 2] and least_m < blue_range: # 終値より始値の方が高く、5個平均よりもレンジが大きいとき
            print('short')

least_m: 0.171 red_range: 0.174 blue_range: 0.022
long


In [125]:
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12609641-001"
    access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    
    params = {'count': 1, 'granularity': 'H12'}
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    tmp_df = to_dataframe(r)

In [126]:
tmp_df

,Time,Volume,Open,High,Low,Close
0,2019-11-21 22:00:00+00:00,3662,108.632,108.708,108.557,108.598


# 今度こそガチ

In [9]:
def order(l_s):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12609641-001"
    access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    kly_rate = 0.5
    # 口座残高の取得
    s = accounts.AccountSummary(accountID)
    api.request(s)
    assets = float(s.response['account']['balance'])
    # 現在のポジションを取得
    p = positions.OpenPositions(accountID=accountID)
    api.request(p)
    try:
        l_pos = int(p.response['positions'][0]['long']['units'])
        s_pos = int(p.response['positions'][0]['short']['units'])
        pos = l_pos + s_pos
    except: # ポジション持ってなかったらエラー吐くから0入れとく
        pos = 0
    # もし同じポジション持ってたら以下全部パス
    if (l_s == 'long' and pos > 0) or (l_s == 'short' and pos < 0):
        pass
    else:
        # もし逆のポジション持ってたら決済
        if pos == 0:
            pass
        else:
            if pos > 0:
                position_data = { "longUnits": "ALL" }
            else:
                position_data = { "shortUnits": "ALL" }
            p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
            api.request(p)
        # 現在価格の取得
        params = {"instruments": "USD_JPY"}
        pi = PricingInfo(accountID=accountID, params=params)
        api.request(pi)
        if l_s == 'long':
            ab = 'asks'
            p_and_n = '+'
        else:
            ab = 'bids'
            p_and_n = '-'
        entry_rate = float(pi.response['prices'][0][ab][0]['price'])
        # ケリー基準から lot の指定
        kly = assets * kly_rate
        lot = floor(kly / entry_rate)
        data = {
          "order": {
            "instrument": "USD_JPY",
            "units": "{}{}".format(p_and_n, lot),
            "type": "MARKET",
            "positionFill": "DEFAULT"
          }
        }
        o = orders.OrderCreate(accountID, data=data)
        api.request(o)
        print("units: {}{}".format(p_and_n, lot))
    
def foo(hour):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12609641-001"
    access_token = "11f2a77cf99d3d29afc4c1cb1a0fb36b-e0fc1d1bf476926157f1d5992466fdf4"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    
    df = pd.DataFrame(columns=['Time', 'Volume', 'Open', 'High', 'Low', 'Close'])
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    
    range_list = []
    # 実行したときの曜日を取得、現在時刻のログを吐く
    now = int(time.time())
    print('現在時刻: {}'.format(datetime.datetime.fromtimestamp(now)))
    now_w = datetime.datetime.fromtimestamp(now - 32400).weekday()
    
    # 土日の処理
    if now_w == 5 or now_w == 6:
        print('土日はお休み')
        pass
    # 土日以外はこっち
    else:
        # データ取得
        params = {'count': 7, 'granularity': 'H12'}
        r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
        api.request(r)
        df = to_dataframe(r)
        # 直近5個 + 1個になるように最新の足を1個落として揃える
        if hour == df.iat[6, 0].hour:
            df.drop(index=6, inplace=True)
            print('計画通り......')
        # 何らかのバグで最新の足取れなかったときはこっち
        else:
            df.drop(index=0, inplace=True)
            df.reset_index(drop=True, inplace=True)
            print('☆☆粉バナナ☆☆')
            
        # 直近5個のレンジ幅の平均取得
        for i in range(5):
            range_ = abs(round(df.iat[i, 3] - df.iat[i, 4], 3)) # |High - Low|
            range_list.append(range_)
        least_m = round(mean(range_list) * 0.5, 3)
        
        red_range = round(df.iat[5, 3] - df.iat[5, 2], 3) # High - Open 赤幅(上げ幅)
        blue_range = round(df.iat[5, 2] - df.iat[5, 4], 3) # Open - Low 青幅(下げ幅)
        print('least_m: {} red_range: {} blue_range: {}'.format(least_m, red_range, blue_range))
        print(df)
        
        # 上昇トレンドと判断しロング
        if df.iat[5, 2] < df.iat[5, 5] and least_m < red_range: # 始値より終値の方が高く、5個平均よりもレンジが大きいとき
            order('long')
            
        # 下降トレンドと判断しショート
        if df.iat[5, 5] < df.iat[5, 2] and least_m < blue_range: # 終値より始値の方が高く、5個平均よりもレンジが大きいとき
            order('short')
    
# ガチトレード
def main():
    schedule.every().day.at("19:00").do(foo, 10)
    schedule.every().day.at("07:00").do(foo, 22)
    
    while True:
        schedule.run_pending()
        time.sleep(1)

In [ ]:
main()

現在時刻: 2019-11-26 19:00:00
☆☆粉バナナ☆☆
least_m: 0.105 red_range: 0.268 blue_range: 0.065
                       Time  Volume     Open     High      Low    Close
0 2019-11-21 10:00:00+00:00    8431  108.526  108.700  108.504  108.612
1 2019-11-21 22:00:00+00:00    5994  108.632  108.708  108.532  108.540
2 2019-11-22 10:00:00+00:00    5563  108.542  108.732  108.478  108.651
3 2019-11-24 22:00:00+00:00    4977  108.673  108.896  108.638  108.856
4 2019-11-25 10:00:00+00:00    3599  108.854  108.978  108.816  108.924
5 2019-11-25 22:00:00+00:00    6205  108.940  109.208  108.875  108.955
現在時刻: 2019-11-27 07:00:00
☆☆粉バナナ☆☆
least_m: 0.118 red_range: 0.188 blue_range: 0.03
                       Time  Volume     Open     High      Low    Close
0 2019-11-21 22:00:00+00:00    5994  108.632  108.708  108.532  108.540
1 2019-11-22 10:00:00+00:00    5563  108.542  108.732  108.478  108.651
2 2019-11-24 22:00:00+00:00    4977  108.673  108.896  108.638  108.856
3 2019-11-25 10:00:00+00:00    3599  10